In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

from socco2011.functions import dimensions, functionssocco
from cec2020.functions import functionscec

In [ ]:
config = {
    "FDA": {
        "c": "black",
        "linestyle": "solid",
        "marker":".",
        "title": "FDA",
    },
    "DIRECT": {
        "c": "green",
        "linestyle": "dotted",
        "marker":".",
        "title": "Direct"},
    "DIRECTL": {
        "c": "lime",
        "linestyle": (0, (1, 10)),
        "marker":".",
        "title": "DIRECTL",
    },
    "DIRECTR": {
        "c": "turquoise",
        "linestyle": "dashdot",
        "marker":".",
        "title": "DIRECTR",
    },
    "SOO": {
        "c": "red",
        "linestyle": "dashed",
        "marker":".",
        "title": "SOO",
    },
    "DIRECTBs": {
        "c": "cyan",
        "linestyle": "solid",
        "marker": "+",
        "title": "DIRECTBs",
    },
    "FDABs": {
        "c": "dimgrey",
        "linestyle": "solid",
        "marker": "x",
        "title": "FDABs",
    },
    "FDADBs": {
        "c": "grey",
        "linestyle": "solid",
        "marker": ">",
        "title": "FDADBs",
    },
    "FDAC": {
        "c": "lightgrey",
        "linestyle": "solid",
        "marker": "*",
        "title": "FDAC",
    },
    "FDAD": {
        "c": "gainsboro",
        "linestyle": "solid",
        "marker":"D",
        "title": "FDAD",
    },
    "SOOBs": {
        "c": "orange",
        "linestyle": "solid",
        "marker": "s",
        "title": "SOOBs",
    },
}

algo_labels = ['FDA','DIRECT','DIRECTL','DIRECTR','SOO','DIRECTBs','FDABs','FDADBs','FDAC','FDAD','SOOBs']

df = pd.DataFrame(columns=["loss", "dimension", "algorithm", "function"])
benchmark = "socco2011"
filename = f"summary_{benchmark}.csv"

if benchmark == "socco2011":
    all_functions = functionssocco
elif benchmark == "cec2020":
    all_functions = functionscec

In [ ]:
info_functions = {
    f.__class__.__name__: {"optimum": f.optimum} for f in all_functions
}

## Read data

In [ ]:
with open(filename, 'a+') as file:
    file.write("dimension,algorithm,function,min,max,mean,std,median,1_quart,3_quart\n")
    
    for f in info_functions:
        for d in dimensions: 
            info_functions[f][f"{d}"] = {}
            for algo in list(config.keys()):       
                
                new = pd.read_csv(
                    f"./experiments_{benchmark}/{algo}/D{d}_{f}_save/outputs/all_evaluations.csv"
                )
                new["loss"]=new["objective"]
                new["dimension"] = d
                new["algorithm"] = algo
                new["function"] = f.__class__.__name__
                new["error"] = new["loss"] - info_functions[f]["optimum"]

                info_functions[f][f"{d}"][algo] = {
                    "min": new["error"].min(),
                    "max": new["error"].max(),
                    "mean": new["error"].mean(),
                    "std": new["error"].std(),
                    "median": new["error"].median(),
                    "1_quart": new["error"].quantile(0.25),
                    "3_quart": new["error"].quantile(0.75),
                }
                
                file.write(f"""{d},{algo},{f},{info_functions[f][f"{d}"][algo]["min"]},{info_functions[f][f"{d}"][algo]["max"]},{info_functions[f][f"{d}"][algo]["mean"]},{info_functions[f][f"{d}"][algo]["std"]},{info_functions[f][f"{d}"][algo]["median"]},{info_functions[f][f"{d}"][algo]["1_quart"]},{info_functions[f][f"{d}"][algo]["3_quart"]}\n""")

# Summary analysis

In [ ]:
data_name = "fullsoco"

datasocco2011 = pd.read_csv("./summary_socco2011.csv")
datacec2020 = pd.read_csv("./summary_cec2020.csv")

datasocco2011["function"].replace(['Schwefel_2_21','Schwefel_2_22','Schwefel_1_2'],
                                  ['Schwefel-2.21','Schwefel-2.22','Schwefel-1.2'], inplace=True)

dataall = datacec2020.append(datasocco2011)

if data_name == "soco":
    data_selec = datasocco2011
elif data_name=="all":
    data_selec = dataall
elif data_name=="cec":
    data_selec = datacec2020
elif data_name=="fullsoco":
    data_selec = datasocco2011
else:
    raise ValueError("Unknown data name")

other = ['F10','CF9F1-25', 'CF9F3-25', 'CF9F4-25', 'CF10F7-25','CF9F1-75', 'CF9F3-75', 'CF9F4_75', 'CF10F7_75']
if data_name =="soco":
    sorter = ['Sphere', 'Schwefel-2.21', 'Rosenbrock', 'Rastrigin', 'Griewank','Ackley', 'Schwefel-2.22', 'Schwefel-1.2', 'Bohachevsky','Schaffer'][:6]
elif data_name=="all":
    sorter = ['Sphere', 'Schwefel-2.21', 'Rosenbrock', 'Rastrigin', 'Griewank','Ackley', 'Schwefel-2.22', 'Schwefel-1.2', 'Bohachevsky','Schaffer'][:6] +[f.__class__.__name__ for f in functionscec]
elif data_name=="cec":
    sorter = [f.__class__.__name__ for f in functionscec]
elif data_name=="fullsoco":
    sorter = ['Sphere', 'Schwefel-2.21', 'Rosenbrock', 'Rastrigin', 'Griewank','Ackley', 'Schwefel-2.22', 'Schwefel-1.2', 'Bohachevsky','Schaffer']


#### Latex table with minimum values

In [ ]:
ok = data_selec[data_selec["dimension"]==100]
df2 = pd.DataFrame(ok[['min','function','algorithm']])
df2 = df2.pivot_table(index='function',columns=['algorithm'],values='min')
df2 = df2.loc[sorter]

# apply some formatting for all numbers (optional)
df_s = df2.style.format("{:.2E}")
# loop through rows and find which column for each row has the highest value
for row in df2.index:
    col = df2.loc[row].idxmin()
    # redo formatting for a specific cell
    df_s = df_s.format(lambda x: "\\textbf{" + f'{x:.2E}' + "}", subset=(row, col))
print(df_s.to_latex())

#### Statistical analysis

In [ ]:
df2 = pd.DataFrame(data_selec[['min','function','dimension','algorithm']])
df2 = df2.pivot_table(index='function',columns=['dimension','algorithm'],values='min')
df2 = df2.loc[sorter]
df2.reindex(columns=algo_labels, level='algorithm')
print(df2)

##### Friedman Chisquare test

In [ ]:
rank = []
q_test = []
for d in dimensions:
    sub1 = df2.loc[:, ([d], algo_labels)]
    sub1.columns = sub1.columns.droplevel()
    rank.append(sub1.loc[sorter[0]].rank())
    for i in sorter[1:]:
        rank[-1] += sub1.loc[i].rank()
    rank[-1] /= len(sorter)
    q_test.append((12*len(sorter))/(11*12)*np.sum((rank[-1]-6)**2))
    
from scipy import stats
for d in dimensions:
    t1 = df2.loc[:, ([d], algo_labels)]
    t1.columns = t1.columns.droplevel()
    groups = []

    for g in algo_labels:
        groups.append(t1[g])
    res = stats.friedmanchisquare(*groups)
    print(res)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

N = 11
ind = np.arange(N) # the x locations for the groups
width = 0.50
fig = plt.figure(figsize=(8, 6))
ax = fig.add_axes([0,0,1,1])

# Get the "viridis" colormap
cmap = plt.get_cmap('tab10')

# Create an array of equally spaced values between 0 and 1
color_values = np.linspace(0, 1, 6)

# Get the corresponding colors from the colormap
colors = [cmap(value) for value in color_values]

ax.bar(ind, rank[0], width, color="#332288",label="D10")
ax.bar(ind, rank[1], width,bottom= rank[0], color="#117733",label="D15")
ax.bar(ind, rank[2], width,bottom= rank[0]+rank[1], color="#44AA99",label="D20")
ax.bar(ind, rank[3], width,bottom= rank[0]+rank[1]+rank[2], color="#88CCEE",label="D30")
ax.bar(ind, rank[4], width,bottom= rank[0]+rank[1]+rank[2]+rank[3], color="#DDCC77",label="D50")
ax.bar(ind, rank[5], width,bottom= rank[0]+rank[1]+rank[2]+rank[3]+rank[4], color="#CC6677",label="D100")

for c in ax.containers:
    # Optional: if the segment is small or 0, customize the labels
    labels = ["{:.2f}".format(v.get_height()) if v.get_height() > 0 else '' for v in c]
    
    # remove the labels parameter if it's not needed for customized labels
    ax.bar_label(c, labels=labels, label_type='center')
    
ax.set_ylabel('Rank sum')
#ax.set_title('Ranked sum for SOCO2011')
ax.set_xticks(ind,  ['FDA','DIRECT','DIRECTL','DIRECTR','SOO','DIRECTBs','FDABs','FDADBs','FDAC','FDAD','SOOBs'])
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels))
plt.savefig(f"{data_name}_rank.png",bbox_inches='tight', dpi=300)
plt.show()
plt.close(fig)

In [ ]:
stat_w = np.zeros((len(dimensions),N,N))
p_value_w = np.zeros((len(dimensions),N,N))
for k,d in enumerate(dimensions):
    t1 = df2.loc[:, ([d], algo_labels)]
    t1.columns = t1.columns.droplevel()
    for i,g1 in enumerate(algo_labels):
        for j,g2 in enumerate(algo_labels):
            if g1 != g2:
                stat_w[k,i,j],p_value_w[k,i,j] = stats.wilcoxon(t1[g1],t1[g2], zero_method='zsplit')
                
stat_f = np.zeros((len(dimensions)))
p_value_f = np.zeros((len(dimensions)))
for k,d in enumerate(dimensions):
    t1 = df2.loc[:, ([d], algo_labels)]
    t1.columns = t1.columns.droplevel()
    groups = [t1[g1] for g1 in algo_labels]
    stat_f[k],p_value_f[k] = stats.friedmanchisquare(*groups)

In [ ]:
from matplotlib.ticker import MultipleLocator
import matplotlib

for i,d in enumerate(dimensions):
    fig, ax = plt.subplots()
    
    cmap = matplotlib.cm.Oranges
    cmap.set_bad(color='white')
    masked_array = np.ma.masked_where(p_value_w[i] < 0.05, p_value_w[i])
    cax = ax.matshow(masked_array, cmap=cmap)
    fig.colorbar(cax)
    for (i, j), z in np.ndenumerate(p_value_w[i]):
        ax.text(j, i, '{:0.1f}'.format(z), ha='center', va='center')


    ax.yaxis.set_major_locator(MultipleLocator(1))
    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.set_xticklabels(['']+algo_labels, rotation=90)
    ax.set_yticklabels(['']+algo_labels, rotation=0)
    ax.set_title(fr"Pair-wise Wilcoxon test on dimension {d} with $\alpha$=5%")
    fig.tight_layout()
    plt.savefig(f"{data_name}_wilc_d{d}.png",bbox_inches='tight', dpi=300)
    plt.show()
    plt.close(fig)

In [ ]:
from matplotlib.colors import ListedColormap
from matplotlib.ticker import MultipleLocator,AutoMinorLocator
import matplotlib.patches as mpatches

for i,d in enumerate(dimensions):
    colors_matrix = np.zeros((len(rank[i]),len(rank[i])))
    greater = np.greater.outer(rank[i].to_numpy(), rank[i].to_numpy())
    colors_matrix = np.where(greater, 1, colors_matrix)
    lower = np.less.outer(rank[i].to_numpy(), rank[i].to_numpy())
    colors_matrix = np.where(lower, -1, colors_matrix)
    colors_matrix = np.where(p_value_w[i]>0.05,0,colors_matrix)

    cmap = ListedColormap(['red', 'lightgrey', 'green'])
    fig, ax = plt.subplots()
    cax = ax.matshow(colors_matrix,cmap=cmap)

    # Define hatches for specific values
    hatch_values = {-1: '.', 1:'x'}

    # Get the current axes
    ax = plt.gca()

    # Loop through the data and add hatches
    for i in range(colors_matrix.shape[0]):
        for j in range(colors_matrix.shape[1]):
            value = colors_matrix[i, j]
            if value in hatch_values:
                ax.add_patch(plt.Rectangle((j - 0.5, i - 0.5), 1, 1, fill=False, hatch=hatch_values[value], edgecolor='black'))



    ax.yaxis.set_major_locator(MultipleLocator(1))
    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.set_xticklabels(['']+algo_labels, rotation=90)
    ax.set_yticklabels(['']+algo_labels, rotation=0)
    ax.set_title(f"Ranks and Pair Wise Wilcoxon test comparison\nfor n={d} on SOCO2011")
    fig.tight_layout()

    # Minor ticks
    minor_locator = AutoMinorLocator(2)
    ax.xaxis.set_minor_locator(minor_locator)
    ax.yaxis.set_minor_locator(minor_locator)


    ax.grid(which='minor', color='grey', linestyle='-', linewidth=1)
    plt.arrow(0, 0, 0, 4, head_width=0, head_length=0, linewidth=1, color='black', length_includes_head=True)
    plt.arrow(0, 4, -0.45, 0, head_width=0.2, head_length=0.2, linewidth=1, color='black', length_includes_head=True)
    plt.annotate('FDA vs SOO',
                 xy=(-0.5, 3),
                 xytext=(10, -10),
                 textcoords='offset points',
                rotation=90)

    patches = [mpatches.Patch(color="lightgrey", label="Unsignificative\n"+r"($\alpha>5\%$)"),
               mpatches.Patch(color="green", label="Better", hatch="xxx"),
               mpatches.Patch(color="red", label="Worse", hatch="...")]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0. )
    plt.savefig(f"{data_name}_compar{d}.png",bbox_inches='tight', dpi=300,facecolor=fig.get_facecolor())
    plt.show()
    plt.close(fig)